In [16]:
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram

import numpy as np
from scipy.optimize import minimize

In [17]:
# dij需求i到设施j的成本
d = [[1, 2], [1, 2]]
n = 2   # 两个设施点
m = 2   # 两个需求点
num_qubits = n + 2 * n * m

# gi设施i的建设成本
g = [2, 1]

In [18]:
GateX = np.array([[0, 1],[1, 0]])
GateY = np.array([[0, -1j],[1j, 0]])
GateZ = np.array([[1, 0],[0, -1]])

# 定义σ+和σ-矩阵
sigma_plus = np.array([[0, 1], [0, 0]])
sigma_minus = np.array([[0, 0], [1, 0]])

def add_in_target(num_qubits, target_qubit, gate=np.array([[1, 0],[0, -1]])):
    H = np.eye(2 ** (target_qubit))
    H = np.kron(H, gate)
    H = np.kron(H, np.eye(2 ** (num_qubits - 1 - target_qubit)))
    return H

def generate_Hp(n, m, d, g):
    # 初始化 Hp 矩阵为零矩阵
    Hp = np.zeros((2**num_qubits, 2**num_qubits))
    for i in range(m):
        for j in range(n):
            Hp += d[i][j] * (add_in_target(num_qubits, n * (1 + i) + j) - np.eye(2**num_qubits)) / 2
    
    for j in range(n):
        Hp +=  g[j] * (add_in_target(num_qubits, j)- np.eye(2**num_qubits)) / 2
    return Hp
        
Hp = generate_Hp(n, m, d, g)


In [19]:
import sys
sys.path.append('../../')
import z_library.linear_system as ls

In [20]:
def gnrt_gate_hdi(u):
    result = []
    nonzero_indices = ls.find_nonzero_indices(u)
    for urow, nrow in zip(u, nonzero_indices):
        # 把非0元素映射成01
        filtered_arr = [0 if x == -1 else 1 for x in urow if x != 0]
        binary_str = ''.join(map(str, filtered_arr))
        # 取到二进制表示的数
        map_num = int(binary_str, 2)
        # print(map_num)
        length = len(nrow)
        scale = 2**length
        matrix = np.zeros((scale, scale))

        matrix[map_num, scale - 1 - map_num] = 1
        map_num = 2**length - 1 - map_num
        # print(map_num)
        matrix[map_num, scale - 1 - map_num] = 1
        result.append(matrix)
    return result

# print("非零元素的索引：", nonzero_indices)

u = ls.gnrt_u(n, m)
print(u)
nonzero_indices = ls.find_nonzero_indices(u)
gate_hds = gnrt_gate_hdi(u)
print(nonzero_indices)

[[-1.  1. -1.  1. -1.  1.  0.  0.  0.  0.]
 [ 0.  0.  1. -1.  0.  0. -1.  1.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  1.  0.  1.  0.]
 [ 0.  1. -1.  1.  0.  0.  1.  0.  0.  1.]]
[[0, 1, 2, 3, 4, 5], [2, 3, 6, 7], [0, 6, 8], [1, 2, 3, 6, 9]]


In [21]:
gnrt_gate_hdi(u)[1]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [22]:
from scipy.linalg import expm
def build_circ(params):
  depth = len(params) // 2
  qc = QuantumCircuit(num_qubits)
  beta = params[:depth]
  gamma = params[depth:]
  for i in [4,6,8]:
    qc.x(i)
  for dp in range(depth):
    qc.unitary(expm(-1j * beta[dp] * Hp), range(num_qubits))
    for gate_hdi, ind in zip(gate_hds, nonzero_indices):
      qc.unitary(expm(-1j * gamma[dp] * gate_hdi), (num_qubits - 1 - i for i in ind))
  qc.measure_all()
  return qc

In [23]:
ls.set_print_form()
build_circ(np.full(2 * 2, np.pi/3)).draw()

┌──────────┐                                    ┌──────────┐»
    q_0: ─────┤0         ├────────────────────────────────────┤4         ├»
              │          │                        ┌──────────┐│          │»
    q_1: ─────┤1         ├────────────────────────┤2         ├┤          ├»
              │          │            ┌──────────┐│          ││          │»
    q_2: ─────┤2         ├────────────┤3         ├┤          ├┤          ├»
              │          │            │          ││          ││          │»
    q_3: ─────┤3         ├────────────┤2         ├┤1         ├┤3         ├»
         ┌───┐│          │┌──────────┐│          ││          ││          │»
    q_4: ┤ X ├┤4         ├┤5         ├┤          ├┤          ├┤  Unitary ├»
         └───┘│  Unitary ││          ││  Unitary ││          ││          │»
    q_5: ─────┤5         ├┤4         ├┤          ├┤  Unitary ├┤          ├»
         ┌───┐│          ││          ││          ││          ││          │»
    q_6: ┤ X ├┤6         ├┤3         ├┤1         ├┤          ├┤2         ├»
         └───┘│          ││  Unitary ││          ││          ││          │»
    q_7: ─────┤7         ├┤2         ├┤0         ├┤          ├┤1         ├»
         ┌───┐│          ││          │└──────────┘│          ││          │»
    q_8: ┤ X ├┤8         ├┤1         ├────────────┤          ├┤0         ├»
         └───┘│          ││          │            │          │└──────────┘»
    q_9: ─────┤9         ├┤0         ├────────────┤0         ├────────────»
              └──────────┘└──────────┘            └──────────┘            »
meas: 10/═════════════════════════════════════════════════════════════════»
                                                                          »
«         ┌──────────┐                                    ┌──────────┐ ░ ┌─┐   »
«    q_0: ┤0         ├────────────────────────────────────┤4         ├─░─┤M├───»
«         │          │                        ┌──────────┐│          │ ░ └╥┘┌─┐»
«    q_1: ┤1         ├────────────────────────┤2         ├┤          ├─░──╫─┤M├»
«         │          │            ┌──────────┐│          ││          │ ░  ║ └╥┘»
«    q_2: ┤2         ├────────────┤3         ├┤          ├┤          ├─░──╫──╫─»
«         │          │            │          ││          ││          │ ░  ║  ║ »
«    q_3: ┤3         ├────────────┤2         ├┤1         ├┤3         ├─░──╫──╫─»
«         │          │┌──────────┐│          ││          ││          │ ░  ║  ║ »
«    q_4: ┤4         ├┤5         ├┤          ├┤          ├┤  Unitary ├─░──╫──╫─»
«         │  Unitary ││          ││  Unitary ││          ││          │ ░  ║  ║ »
«    q_5: ┤5         ├┤4         ├┤          ├┤  Unitary ├┤          ├─░──╫──╫─»
«         │          ││          ││          ││          ││          │ ░  ║  ║ »
«    q_6: ┤6         ├┤3         ├┤1         ├┤          ├┤2         ├─░──╫──╫─»
«         │          ││  Unitary ││          ││          ││          │ ░  ║  ║ »
«    q_7: ┤7         ├┤2         ├┤0         ├┤          ├┤1         ├─░──╫──╫─»
«         │          ││          │└──────────┘│          ││          │ ░  ║  ║ »
«    q_8: ┤8         ├┤1         ├────────────┤          ├┤0         ├─░──╫──╫─»
«         │          ││          │            │          │└──────────┘ ░  ║  ║ »
«    q_9: ┤9         ├┤0         ├────────────┤0         ├─────────────░──╫──╫─»
«         └──────────┘└──────────┘            └──────────┘             ░  ║  ║ »
«meas: 10/════════════════════════════════════════════════════════════════╩══╩═»
«                                                                         0  1 »
«                                 
«    q_0: ────────────────────────
«                                 
«    q_1: ────────────────────────
«         ┌─┐                     
«    q_2: ┤M├─────────────────────
«         └╥┘┌─┐                  
«    q_3: ─╫─┤M├──────────────────
«          ║ └╥┘┌─┐               
«    q_4: ─╫──╫─┤M├───────────────
«          ║  ║ └╥┘┌─┐            
«    q_5: ─╫──╫──╫

In [24]:
def cost_function(x):
  num = [int(char) for char in x]
  C = 0
  for i in range(m):
    for j in range(n):
      C += d[i][j] * num[n * (1 + i) + j]
      
  for j in range(n):
    C += g[j] * num[j]
  return C

def compute_expectation(counts):
  EV = 0
  total_count = 0
  for x, count in counts.items():
    C = cost_function(x)
    EV += C*count
    total_count += count

  return EV/total_count


def expectation_from_sample(shots = 2000):
  backend = Aer.get_backend('qasm_simulator')
  backend.shots = shots

  def execute_circ(theta):
    qc = build_circ(theta)
    counts = backend.run(qc, seed_simulator=10, shots=shots).result().get_counts()
    return compute_expectation(counts)
  
  return execute_circ

In [25]:
# 初始化迭代计数器
iteration_count = 0
def test(par):
  global iteration_count
  iteration_count = 0
  expectation = expectation_from_sample()
  def callback(x):
      global iteration_count
      iteration_count += 1
      # if iteration_count % 10 == 0:
      #     print(f"Iteration {iteration_count}, Result: {expectation(x)}")
  # 设定最大迭代次数
  max_iterations = 1000

  # 使用 COBYLA 方法进行最小化，并设置 callback 函数
  res = minimize(expectation, par, method='COBYLA', options={'maxiter': max_iterations}, callback=callback)
  # 输出最终结果
  # print("Final Result:", res)
  backend = Aer.get_backend('aer_simulator')a
  backend.shots = 100000
  print(sum(res.x))
  shots=100000
  qc_res = build_circ(params=res.x)
  counts = backend.run(qc_res, seed_simulator=10, shots = shots).result().get_counts()
  # plot_histogram(counts)
  sorted_counts = sorted(counts, key=counts.get, reverse=True)
  for x in sorted_counts[:20]:
    if cost_function(x) == 4:
      print("迭代次数", iteration_count, "{:.2f}%".format(counts[x] / shots * 100), cost_function(x))
      break

In [26]:
dep = 2
for denominator in range(1,10):
  print(f'denominator == {denominator}')
  test(np.full(dep * 2, np.pi/denominator))
  print()

denominator == 1
14.183857021782654
迭代次数 44 99.95% 4

denominator == 2
7.737425801492412
迭代次数 51 100.00% 4

denominator == 3
5.986381094109074
迭代次数 97 98.86% 4

denominator == 4
3.234074686430887
迭代次数 49 99.98% 4

denominator == 5
3.9833241516745095
迭代次数 45 99.99% 4

denominator == 6
3.7346163161326174
迭代次数 48 99.81% 4

denominator == 7
3.6075363364615174
迭代次数 40 99.98% 4

denominator == 8
3.344758397657568
迭代次数 45 99.95% 4

denominator == 9
3.4270607459384093
迭代次数 40 99.95% 4

